# Web scraper de información de vehículos usados

In [103]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm, trange
import time

# Mercado Libre

In [86]:
def get_car_info(url):
    soup = BeautifulSoup(requests.get(url).text, 'lxml')
    car_info = {}
    try:
        car_info['Precio'] = soup.find('div', class_='ui-pdp-price__second-line').find('span', class_='andes-visually-hidden').text.split(' ')[0]
        car_info['Precio'] = int(car_info['Precio'])
        table = soup.find('table', class_='andes-table')
        for tr in table.find_all('tr'):
            key = tr.find('th').text
            value = tr.find('td').text
            car_info[key] = value
        
        car_info['Kilómetros'] = int(car_info['Kilómetros'].replace('km', ''))
        return car_info
    except:
        return get_car_info(url)

In [88]:
html_text = requests.get('https://carros.mercadolibre.com.co/').text
soup = BeautifulSoup(html_text, 'lxml')

In [95]:
num_pages = int(soup.find('li', class_='andes-pagination__page-count').text.split(' ')[-1])
num_pages

42

In [ ]:
df = pd.DataFrame()
for i in trange(1,num_pages+1):
    html_text = requests.get(f'https://carros.mercadolibre.com.co/_Desde_{(i*48)+1}').text
    soup = BeautifulSoup(html_text, 'lxml')
    cars = soup.find_all('div', class_='ui-search-result__wrapper shops__result-wrapper')
    for car in cars:
        url = car.find('a', class_='ui-search-link')['href']
        df = pd.concat([df, pd.DataFrame([get_car_info(url)])])


df.reset_index(drop=True, inplace=True)

In [98]:
df

,Precio,Marca,Modelo,Año,Versión,Color,Tipo de combustible,Puertas,Transmisión,Motor,Tipo de carrocería,Kilómetros
0,32500000,Volkswagen,Jetta,2011,2.0 Europa,Gris,Gasolina,4,Mecánica,2.0,Sedán,166000
1,34000000,Chevrolet,Captiva,2010,3.6 Sport,Gris,Gasolina,4,Automática,3.6,NaN,73000
2,125000000,Mazda,CX-5,2020,2.0 Touring Camioneta,Gris,Gasolina,5,Automática,2.0,Camioneta,25000
3,170000000,Honda,CR-V,2022,1.5 Exl,Gris,Gasolina,5,Automática,1.5,Camioneta,7000
4,26900000,Chevrolet,Aveo,2010,1.6 Gti Limited,Plateado,Gasolina,3,Mecánica,1.6,Coupé,105000
...,...,...,...,...,...,...,...,...,...,...,...,...
1963,65000000,Mazda,3,2017,PRIME,Plateado,Gasolina,4,Mecánica,2.000,Sedán,55917
1964,44500000,Chevrolet,Captiva,2014,Gt,Blanco,Gasolina,4,Automática,2.400,Station Wagon,94000
1965,129800000,Nissan,Frontier,2020,NP 300,Azul,Gasolina,4,Mecánica,NaN,NaN,40000
1966,81000000,Hyundai,Tucson,2016,2.0 Gl Advance,Rojo,Gasolina,5,Mecánica,2.0,Camioneta,46400


# Vende tu nave

In [126]:
import json

In [134]:
html_text = requests.get('https://www.vendetunave.co/vehiculos/carrosycamionetas?&estado=Usado').text
soup = BeautifulSoup(html_text, 'lxml')
data = json.loads(soup.find('script', id='__NEXT_DATA__').text)
data = data['props']['pageProps']['data']['vehicles']

print(data[0])

# for car in data:
#     # car['price'] = int(car['price'].replace('$', '').replace('.', ''))
#     print(car)

{'id': 28706, 'tipo_moto': 0, 'title': 'KIA RÍO UB EX 2013 AT', 'descripcion': 'Automático \nVersión full equipo\nVidrios eléctricos \nEspejos retrovisores eléctricos \nSun roof \n95.600 km \nSoat hasta 05/09/23\nTecno hasta 14/08/23\nCambio de aceite reciente \nCambio de pastillas reciente ', 'precio': 43000000, 'condicion': 'Usado', 'cilindraje': '1400', 'ano': 2013, 'kilometraje': 95600, 'placa': '5', 'fecha_publicacion': '2023-02-28 00:00:05', 'labelCiudad': 'MEDELLÍN', 'nameImage': '63d316b3cd1a8', 'extension': 'jpeg', 'new_image': 2, 'modelo': 'Rio', 'marca': 'KIA', 'combustible': 'Gasolina', 'transmision': 'Automática', 'tipoPrecioLabel': 'Negociable', 'tipoLabel': 'Carros y camionetas', 'financiacion': 0, 'confiable': 0, 'blindado': 0, 'permuta': 0, 'promocion': 0, 'peritaje': '0', 'premium': 1, 'active_premium': '2023-01-27 01:27:10'}
